# Noise scaling by probabilistic error amplification (PEA)

Here we scale the noise by scaling the noise level (probability) of inserting operations. For a "perfect" noise model and assuming zero sampling error, in the limit of infinite samples these operations would completely cancel the noise, but by scaling up the noise level of the representations of noisy gates, we instead amplify the noise. 

In [1]:
from functools import partial
import numpy as np
from typing import List
from cirq import DensityMatrixSimulator, Circuit, depolarize
from mitiq import QPROGRAM, Executor
from mitiq.benchmarks import generate_rb_circuits
from mitiq.pec import represent_operations_in_circuit_with_local_depolarizing_noise, sample_circuit
from mitiq.zne import  RichardsonFactory, LinearFactory, ExpFactory

## Utilities for PEA scaling

We first define some functions to evaluate the quasi-probability distribution at each noise scale factor.

In [2]:
def generate_scaled_circuits(ideal_circuit: QPROGRAM, scale_factors: List[float], epsilon: float) -> List[QPROGRAM]:
    """Returns a list of represented circuits associated to the input array of scale factors."""
    
    # This is based on the Lagrange interpolation formula
    scaled_circuits = []
    for s in scale_factors:
        scaled_circuits.append(represent_operations_in_circuit_with_local_depolarizing_noise(ideal_circuit, s * epsilon))

    return scaled_circuits

In [3]:
rnd_state = np.random.RandomState(0)  # Set a seed

## The error mitigation problem

Let us define a function which executes a circuit with depolarizing noise and returns an expectation value.

In [4]:
def ideal_executor(circ: Circuit) -> float:
    """Simulates a circuit without noise and returns the expectation value
    of the projector |00...><00...|.
    """
    rho = DensityMatrixSimulator().simulate(circ).final_density_matrix
    return np.real(rho[0, 0]) 

def noisy_executor(circ: Circuit, noise_level, shot_noise=0) -> float:
    """Simulates a circuit with depolarizing noise and returns the expectation value
    of the projector |00...><00...|.
    """
    noisy_circuit = circ.with_noise(depolarize(noise_level))
    return ideal_executor(noisy_circuit) + rnd_state.normal(scale=shot_noise)

Let us first define a simple RB circuit (ideally equal to the identity):

In [5]:
ideal_circuit = generate_rb_circuits(n_qubits=1, num_cliffords=5)[0]

print("Circuit under consideration: \n", ideal_circuit)
print("Circuit under consideration (operations): \n", list(ideal_circuit.all_operations()))
print("Number of gates:", len(list(ideal_circuit.all_operations())))
print("Ideal expectation value:", ideal_executor(ideal_circuit))
print("Noisy expectation value:", noisy_executor(ideal_circuit, noise_level=0.1))

Circuit under consideration: 
 0: ───Y───X^-0.5───X^-0.5───Y^0───Y^0.5───X^-0.5───Y───X^0───X^-0.5───Y^-0.5───X───Y^0───
Circuit under consideration (operations): 
 [cirq.Y(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), (cirq.Y**0.0).on(cirq.LineQubit(0)), (cirq.Y**0.5).on(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), cirq.Y(cirq.LineQubit(0)), (cirq.X**0.0).on(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), (cirq.Y**-0.5).on(cirq.LineQubit(0)), cirq.X(cirq.LineQubit(0)), (cirq.Y**0.0).on(cirq.LineQubit(0))]
Number of gates: 12
Ideal expectation value: 1.0
Noisy expectation value: 0.5897836685180664


For reproducibility, we hard code one of the random circuits produced by the previous cell.

In [6]:
ideal_circuit = Circuit([(cirq.X**0.5).on(cirq.LineQubit(0)), (cirq.Y**-0.5).on(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), (cirq.Y**0.0).on(cirq.LineQubit(0)), (cirq.X**-0.5).on(cirq.LineQubit(0)), (cirq.Y**0.5).on(cirq.LineQubit(0)), (cirq.X**0.5).on(cirq.LineQubit(0)), (cirq.Y**0.0).on(cirq.LineQubit(0)), (cirq.Y**-0.5).on(cirq.LineQubit(0)), (cirq.X**0.5).on(cirq.LineQubit(0)), (cirq.Y**-0.5).on(cirq.LineQubit(0)), (cirq.Y**0.5).on(cirq.LineQubit(0)), (cirq.X**0.5).on(cirq.LineQubit(0)), (cirq.Y**0.5).on(cirq.LineQubit(0))])
print("Circuit under consideration: \n", ideal_circuit)
print("Circuit under consideration (operations): \n", list(ideal_circuit.all_operations()))
print("Number of gates:", len(list(ideal_circuit.all_operations())))
print("Ideal expectation value:", ideal_executor(ideal_circuit))
print("Noisy expectation value:", noisy_executor(ideal_circuit, noise_level=0.1))

NameError: name 'cirq' is not defined

## Evaluate at specified noise scale factors and try different extrapolation methods

TODO: debug strange results

In [ ]:
scale_factors = [1, 3, 5, 7]
scaled_circuit_reps = generate_scaled_circuits(ideal_circuit, scale_factors, epsilon=0.05)

scaled_results = []
for circuit_reps in scaled_circuit_reps:
    sampled_circuits, signs, norm = sample_circuit(ideal_circuit, circuit_reps, random_state=rnd_state, num_samples=50,)
    noisy_execute = partial(noisy_executor, noise_level = 0.05)
    executor = Executor(noisy_execute)
    results = executor.evaluate(sampled_circuits, force_run_all=False)
    scaled_results.append( [norm * s * val for s, val in zip(signs, results)])

print(LinearFactory(scale_factors).extrapolate(scale_factors, scaled_results))


[  5878.47204188 -12853.64182268  -6915.02213355  12673.12681408
   6323.09180917  12649.8775832  -12860.84327005 -11999.30828589
 -12423.1504578  -12424.31308024 -11984.4187845    6744.31534865
  12155.47612879 -12059.00405673   6397.2674155   -6469.34585305
 -12829.70688901  12169.96794703  12164.46163181   6761.18300759
   5833.52644101  -6039.96535747  11912.13951882  -6064.17482793
  -6594.00931971   6667.0034694    6370.96508375  12647.12199779
 -12042.7403315   -6899.75860747   6364.78017489   6366.36511181
   5858.43344581   6383.33512783   6692.95289302 -12394.64133786
 -12758.31801426   5932.19211577  12608.73876951  -6087.64492271
   6368.47123325  12169.46782746  -6138.84096235 -12334.66171021
  -6467.6953855    6330.7262836   -6576.89981679   6722.80225312
 -12423.97313491   5821.27036642]


## Test PEA at different noise levels (TODO)

## Compare with folding ZNE and PEC (TODO)

## Visualize data (TODO)